In [13]:
import pandas as pd
import json

In [14]:
f = open('../data/labeling-45a_annotations complete.json')
js = json.load(f)

In [15]:
js.keys()

dict_keys(['id', 'examples', 'schema', 'dataset', 'relations', 'name', 'annotators_per_example'])

In [16]:
del js['id']
del js['relations']
del js['annotators_per_example']
del js['dataset']
del js['name']

In [17]:
tag_types = []
for tag in js['schema']['tags']:
    tag_types.append(tag['name'])
del js['schema']
print(tag_types)

['GoodsDescription', 'GoodsOrigin', 'HSCode', 'Incoterms', 'Quantity', 'Tolerance', 'UnitPriceAmount']


In [18]:
def replace_dots(x):
    return x.replace(':',' ')

def replace_comma(x):
    return x.replace(',','')

In [19]:
# example
content = 'EXW QTY: 192 UNITS CONCENTRATE 7-UP, AS PER BENEFICIARY\'S SALES QUOTATION NO. 790005454 DATED 02.01.2020'
value = 'CONCENTRATE 7-UP,'
tags = ['O' for i in range(len(content.split()))]
for idx, word in enumerate(content.split()):
    if word in value:
        tags[idx] = 'TAG'

print(tags)

['O', 'O', 'O', 'O', 'TAG', 'TAG', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O']


In [20]:
# create DataFrame
df = pd.DataFrame(columns=['words', 'sentence #', 'tag'])

In [21]:
for i in range(len(js['examples'])):
    # create df to append
    df_app = pd.DataFrame(columns=['words', 'sentence #', 'tag'])

    # remove commas and :
    content = replace_dots(replace_comma(js['examples'][i]['content']))

    # create a list with all the annotations
    annotations = js['examples'][i]['annotations']

    #create a list with all tags
    tags = ['O' for i in range(len(content.split()))]

    # for each annotation get the value and add its tag
    for annotation in annotations:
        value = annotation['value']
        for idx, word in enumerate(content.split()):
            if word in value:
                tags[idx] = annotation['tag']

    # create the DataFrame to append
    df_app['words'] = content.split()
    df_app['sentence #'] = i
    df_app['tag'] = tags
    df = pd.concat([df, df_app], ignore_index=True)

In [22]:
df

,words,sentence #,tag
0,CPT,0,Incoterms
1,LAHORE,0,Incoterms
2,AIRPORT,0,Incoterms
3,PAKISTAN,0,Incoterms
4,QTY,0,O
...,...,...,...
60335,INVOICE,1619,O
60336,NO.,1619,O
60337,900659,1619,O
60338,DATED,1619,O


In [23]:
df.to_csv('../data/dataset_from_json.csv', index=False)

# ISSUES
One malfunction that might appear is random words.

In this example
`CPT LAHORE AIRPORT, PAKISTAN PHARMACEUTICAL ALLOPATHIC RAW MATERIAL OF PHARMACEUTICAL GRADE: QTY:100 KGS TRAMADOL HCL BP/EP AT USD: 45 PER KG. AS PER BENEFICIARY'S PROFORMA INVOICE NO. SLS235/2019 DATED: 22-11-2019`

<b> PHARMACEUTICAL ALLOPATHIC RAW MATERIAL</b> is considered <i> GoodsDescription</i>, but the word `PHARMACEUTICAL` appears later as a separate word. We then need to deal with such separate words.

One way to do that is check how many such apparititions there are.

In [24]:
# example from before

# data = js['content'] #original data
# data = data.split()
#
# df = pd.DataFrame(data, columns=['word'])
# df['tag'] = 'O'
# df.head()
#
#
# str = 'FOB'
#
# def change_tag(str, annotations):
#     for ann in annotations:
#         if str in ann:
#             return annotations[ann]
#     return 'O'
#
# change_tag(str, annots)
#
# df['tag'] = df['word'].apply(change_tag, args=[annots])
# df